# Train U-Net for cervix/os detection

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import datetime
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
# Project
import sys
sys.path.append(os.path.join(os.path.abspath(os.path.dirname('.')), '..', 'common'))
from data_utils import type_1_ids, type_2_ids, type_3_ids, test_ids
from data_utils import get_annotations
from metrics import logloss_mc

In [4]:
from data_utils import RESOURCES_PATH, GENERATED_DATA

In [5]:
sloth_annotations_filename = os.path.join(RESOURCES_PATH, 'cervix_os.json')
annotations = get_annotations(sloth_annotations_filename)

In [17]:
import os
import numpy as np
np.random.seed(2017)

from data_utils import RESOURCES_PATH, GENERATED_DATA, get_annotations
from training_utils import get_trainval_id_type_lists2
sloth_annotations_filename = os.path.join(RESOURCES_PATH, 'cervix_os.json')
annotations = get_annotations(sloth_annotations_filename)

train_id_type_list, val_id_type_list = get_trainval_id_type_lists2(annotations=annotations, val_split=0.25)

print "Total : %s, Train : %s, Val : %s" % (len(annotations), len(train_id_type_list), len(val_id_type_list))

Total : 208, Train : 156, Val : 52


In [52]:
from unet_keras_v1 import get_unet
unet = get_unet(input_shape=(3, 224, 224), n_classes=2)
unet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
convolution2d_47 (Convolution2D) (None, 32, 224, 224)  896         input_4[0][0]                    
____________________________________________________________________________________________________
batchnormalization_47 (BatchNorma(None, 32, 224, 224)  64          convolution2d_47[0][0]           
____________________________________________________________________________________________________
activation_47 (Activation)       (None, 32, 224, 224)  0           batchnormalization_47[0][0]      
___________________________________________________________________________________________

In [33]:
from qsub_utils import submit_job
from qsub_utils import setup_configuration
from qsub_utils import PBS_CONFIGURATION

In [34]:
setup_configuration(nodes='1:knl7210:ram96gb')

In [57]:
import time 

unet_cervix_os_detection_with_keras_cmd = [
    "python",
    os.path.abspath(os.path.join("..", "scripts", "unet_cervix_os_detection_with_keras.py"))
]

process, job_info = submit_job(unet_cervix_os_detection_with_keras_cmd, 
                               name='unet_cervix_os_detection', 
                               cwd=RESOURCES_PATH)

try:
    while True:
        out = process.stdout.readline()    
        if len(out) > 0:        
            print out

        if process.poll() is not None and len(out) == 0:
            break
except KeyboardInterrupt:
    !qdel {job_info['id']}
    time.sleep(1.0)
    !qstat

qsub: job 4694.c001 ready





  ########################################################################

  # Colfax Cluster - https://colfaxresearch.com/

  #      Date:           Tue Mar 28 08:42:35 PDT 2017

  #    Job ID:           4694.c001

  #      User:           u2459

  # Resources:           neednodes=1:knl7210:ram96gb,nodes=1:knl7210:ram96gb,walltime=24:00:00

  ########################################################################

  

Using Theano backend.



 2017-03-28 08:42:50.323444 - Get train/val lists ...

Total : 208, Train : 156, Val : 52



 2017-03-28 08:42:50.377709 - Get U-Net model ...



 2017-03-28 08:42:57.689485 - Start training ...

('Training parameters: ', 8, 10, 512, 128)

Load existing file: /home/u2459/Intel_MobileODT/input/generated/os_cervix_stats.npz

No need to recompute statistics

Epoch 1/10

Error in file [/home/u2459/.theano/compiledir_Linux-3.10-el7.xppsl_1.5.0.4036.x86_64-x86_64-with-centos-7.2.1511-Core-x86_64-2.7.12-64/tmpewrysa/mod.

In [48]:
!qstat

In [45]:
#!{" ".join(unet_cervix_os_detection_with_keras_cmd)}

Using Theano backend.
WARNING (theano.tensor.blas): We did not found a dynamic library into the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.

 2017-03-28 06:58:30.556073 - Get train/val lists ...
Total : 208, Train : 156, Val : 52

 2017-03-28 06:58:30.577475 - Get U-Net model ...

 2017-03-28 06:58:32.762387 - Start training ...
('Training parameters: ', 16, 10, 2048, 1024)
Load existing file: /home/u2459/Intel_MobileODT/input/generated/os_cervix_stats.npz
No need to recompute statistics
